In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import random
from dataset import *
from model import *
import gc


In [2]:
class Args:
    def __init__(self):
        self.gpu = 3
        self.run_label = 0
        self.batch_size = 8
        self.learning_rate = 1e-4
        self.n_sample = 256
        self.n_future = 10
        self.n_past = 32
        self.x_dim = 2
        self.y_dim = 131
        self.feat_size = 131
        self.lstm_hidden_dim = 256
        self.lstm_layers = 2
        self.verbose = False
        
args = Args()
device = torch.device('cuda:%d' % args.gpu)
args.device = device

In [3]:
while True:
    args.log_dir = '/data/hdim-forecast/log4/pred/seq=%d/%d-ns=%d-feat_size=%d-bs=%d-lr=%.5f-run=%d' % \
        (args.n_past, args.n_future, args.n_sample, args.feat_size, 
         args.batch_size, args.learning_rate, args.run_label)
    if not os.path.isdir(args.log_dir):
        os.makedirs(args.log_dir)
        break
    args.run_label += 1
print("Run number = %d" % args.run_label)
writer = SummaryWriter(args.log_dir)
log_writer = open(os.path.join(args.log_dir, 'results.txt'), 'w')

start_time = time.time()
global_iteration = 0
random.seed(args.run_label)  # Set a different random seed for different run labels
torch.manual_seed(args.run_label)
    
def log_scalar(name, value, epoch):
    writer.add_scalar(name, value, epoch)
    log_writer.write('%f ' % value)
    
def message(epoch):
    print("Finished epoch %d, time elapsed %.1f" % (epoch, time.time() - start_time))
    
def maybe_print(str_to_print):
    if self.verbose:
        print(str_to_print)

Run number = 0


In [4]:
train_dataset = TrafficDataset(train=True, max_len=args.n_past+args.n_future)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4)

In [5]:
class FeatureNetMoment(nn.Module):
    def __init__(self, x_dim):
        super(FeatureNetMoment, self).__init__()
    
    def forward(self, x):
        first_order = x 
        return first_order
        # second_order = x.unsqueeze(-1).repeat(1, 1, x.shape[1]) * x.unsqueeze(-2).repeat(1, x.shape[1], 1)
        # ind = torch.triu_indices(x.shape[1], x.shape[1]) 
        # second_order = torch.stack([second_order[i][ind[0], ind[1]] for i in range(x.shape[0])])
        # return torch.cat([first_order, second_order.view(x.shape[0], -1)], dim=1)
        

In [6]:
feat_model = FeatureNetMoment(args.feat_size)

In [7]:
feat_model(torch.tensor([[0.1, 0.2], [0.3, 0.5]]))

tensor([[0.1000, 0.2000],
        [0.3000, 0.5000]])

In [8]:
'''Defines the neural network, loss function and metrics'''

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class PredictorRecurrent(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(PredictorRecurrent, self).__init__()
        self.params = params
        self.lstm = nn.LSTM(input_size=params.x_dim+params.y_dim,
                            hidden_size=params.lstm_hidden_dim,
                            num_layers=params.lstm_layers,
                            bias=True,
                            batch_first=False)
        
        # initialize LSTM forget gate bias to be 1 as recommanded by http://proceedings.mlr.press/v37/jozefowicz15.pdf
        for names in self.lstm._all_weights:
            for name in filter(lambda n: "bias" in n, names):
                bias = getattr(self.lstm, name)
                n = bias.size(0)
                start, end = n // 4, n // 2
                bias.data[start:end].fill_(1.)

        self.fc1 = nn.Linear(params.lstm_hidden_dim * params.lstm_layers, 1024)
        self.fc2 = nn.Linear(1024, params.feat_size)
        

    def forward(self, x, y_prev):
        '''
        Predict mu and sigma of the distribution for z_t.
        Args:
            x: ([batch_size, n_past+1, x_dim]), the input feature
            y: ([batch_size, n_past, y_dim]), the previous step label
        Returns:
            pred 
            mu ([batch_size, y_dim]): estimated mean of z_t
            sigma ([batch_size, y_dim]): estimated standard deviation of z_t
            hidden ([lstm_layers, batch_size, lstm_hidden_dim]): LSTM h from time step t
            cell ([lstm_layers, batch_size, lstm_hidden_dim]): LSTM c from time step t
        '''
        hidden, cell = self.init_hidden(x.shape[0]), self.init_cell(x.shape[0])
        
        lstm_input = torch.cat([x, y_prev], dim=-1)
        # print(lstm_input.shape)
        output, (hidden, cell) = self.lstm(lstm_input.permute(1, 0, 2), (hidden, cell))
        
        # print(output.shape, hidden.shape, cell.shape)
            
        # use h from all three layers to calculate mu and sigma
        hidden_permute = hidden.permute(1, 0, 2).contiguous().view(hidden.shape[1], -1) 
        # print(hidden_permute.shape)
        fc = F.leaky_relu(self.fc1(hidden_permute))
        return self.fc2(fc)

    def init_hidden(self, input_size):
        return torch.zeros(self.params.lstm_layers, input_size, self.params.lstm_hidden_dim, device=self.params.device)

    def init_cell(self, input_size):
        return torch.zeros(self.params.lstm_layers, input_size, self.params.lstm_hidden_dim, device=self.params.device)


In [9]:
predictor = PredictorRecurrent(args).to(device)
exp_optim = optim.Adam(predictor.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.StepLR(exp_optim, 20, 0.9)

In [10]:

def next_time(week, num_steps):
    start_time = week.item() * 3600 * 24 * 7
    time_new = torch.linspace(start_time, start_time + 5*60*(num_steps-1), num_steps, device=week.device)
    hour_of_day = (time_new % (3600 * 24)) / (3600. * 24)
    week_day = (time_new % (3600 * 24 * 7)) / (3600 * 24. * 7)
    return torch.stack([hour_of_day, week_day], dim=-1)

In [11]:
bx, by = iter(train_loader).next()
print(bx.shape, by.shape)

torch.Size([8, 42, 2]) torch.Size([8, 42, 131])


In [12]:
# Learn the conditional expectation
for epoch in range(2000):
    for idx, data in enumerate(train_loader):
        exp_optim.zero_grad()

        bx, by = data[0].to(device), data[1].to(device)
        # by has shape [batch_size, seq_len, num_streets]
        actual_feat = feat_model(by[:, -1, :]).detach()
        # print(bx.shape, by.shape)
        pred_exp = predictor(bx[:, :args.n_past], by[:, :args.n_past])
        
        loss_l2 = (actual_feat - pred_exp).pow(2).mean()
        loss_l2.backward()

        writer.add_scalar('loss_l2', loss_l2, global_iteration)
        exp_optim.step()
        global_iteration += 1
            
    scheduler.step()
    message(epoch)

    if (epoch+1) % 10 == 0:
        torch.save(predictor.state_dict(), 'pretrained/predictor_traffic_%d-%d-%s.pt' % (args.feat_size, args.n_future, args.predictor_model))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-b5f2a1af8748>", line 4, in <module>
    exp_optim.zero_grad()
NameError: name 'exp_optim' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.p

NameError: name 'exp_optim' is not defined

In [ ]:
print(actual_feat.shape)